## Exploratory Data Analysis
Several Columns which are redundant in the data.

below is a list of columns which can be representative of the other mentioned columns along with it. 

date_of_birth : dob_year, dob_month
age_upon_intake(days): age_upon_intake,age_upon_intake(years),age_upon_intake_age_group
intake_datetime: converted to unixtime_intake_datetime , [intake_month	intake_year	intake_monthyear intake_hour] are redundant
time_in_shelter_days: time_in_shelter, age_upon_outcome(combnation of both age_upon_intake and time_in_shelter)
[age_upon_outcome_(days)	age_upon_outcome_(years)	age_upon_outcome_age_group	outcome_datetime	outcome_month	outcome_year	outcome_monthyear outcome_hour]
these are also redundant.
outcome_number same as intake_number

count is constant
removing all the unnecessary columns 14 columns remain except the target variable : outcome_type

training and test sets have different data.
breed in test set has 308 breed names which are not there at training set.
for color it is 86.

Hence removing these two variables leads to 12 variables.
animal_id_outcome was primarily considered as a variable.

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv("train.csv")

l = df['animal_id_outcome'].tolist()
print(len(l))
print(len(df))

a1 = []
a2 = []
a3 = []
a4 = []
a5 = []
a6 = []
a7 = []
count = 0 
for animal_id in l:
    a1.append(animal_id[0])
    a2.append(animal_id[1])
    a3.append(animal_id[2])
    a4.append(animal_id[3])
    a5.append(animal_id[4])
    a6.append(animal_id[5])
    a7.append(animal_id[6])

df.insert(1,"animal_id_1", a1)
arr = (a1,a2,a3,a4,a5,a6,a7)
for i in range(2,8):
    df.insert(i,"animal_id_{}".format(i),arr[i-1])

47803
47803


But these variables didn't have much impact on the model performance hence removed.

best performing model is adaboost and random forest ensembled model with accuracy score of 63% in test set.
source codes have been attached below.

### Processing of the data

In [ ]:
import numpy as np
import pandas as pd
import time
from time import strptime
from datetime import datetime
import json
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from sklearn.metrics import precision_score, recall_score, accuracy_score

df = pd.read_csv("train.csv")

def convertunix(a):
    mytm= (a)
    fmt = ("%m/%d/%Y %H:%M")
    epochDate = int(time.mktime(time.strptime(mytm, fmt)))
    return epochDate

df['unixtime_birth_day'] = df.apply(lambda row: convertunix(row.date_of_birth), axis = 1)
df['unixtime_intake_datetime'] = df.apply(lambda row: convertunix(row.intake_datetime), axis = 1)


df = df.drop(["count","dob_year",
              "dob_month","age_upon_intake",
              "age_upon_intake_(years)","age_upon_intake_age_group",
              "intake_datetime","intake_month","intake_year",
              "intake_monthyear","intake_hour","time_in_shelter",
              "age_upon_outcome","date_of_birth","age_upon_outcome_(days)",
              "age_upon_outcome_(years)","age_upon_outcome_age_group",
              "outcome_datetime","outcome_month","outcome_year",
              "outcome_monthyear","outcome_hour","outcome_number","animal_id_outcome"], axis = 1)
df = df.dropna()
# converting categorical to numeric variables.

def category_dict_creation(column_name):
    dictionary = {}
    categorial_outcome = df[column_name].unique().tolist()
    for i in range(len(categorial_outcome)):
        dictionary[categorial_outcome[i]] = i
    return dictionary

def map_values_to_list(df1,dictionary,column_name):
    dfl = df1.copy()
    categorial_outcome = df1[column_name].unique().tolist()
    outcome_list = []
    for i in dfl[column_name]:
        outcome_list.append(dictionary[i])
    dfl.insert(1,column_name+"_int",outcome_list)
    dfl = dfl.drop(column_name , axis = 1)
    return dfl

def convert_category_int(df,dictionary):
    dfl = df.copy()
    dtypes = dfl.dtypes
    for colname in dfl.columns:
        if dtypes[colname] == "object" :
#             dictionary = category_dict_creation(colname)
            dfl = map_values_to_list(dfl,dictionary[colname],colname)
    return dfl

dtypes = df_target.dtypes
dict_of_dicts = {}
for colname in df_target.columns:
    if dtypes[colname] == "object":
        dictionary = category_dict_creation(colname)
        dict_of_dicts[colname] = dictionary

df1 = convert_category_int(df_target,dict_of_dicts)
with open('categorical_variable_mapping.txt', 'w') as outfile:
    json.dump(dict_of_dicts, outfile)

target = pd.DataFrame(df["outcome_type"])
target_int = pd.DataFrame(df1["outcome_type_int"])
X = df1.drop(["outcome_type_int","color_int","breed_int"], axis = 1)

target.to_csv("target.csv")
target_int.to_csv("target_int.csv")
X.to_csv("processed_final_data.csv")


## Model Selection

### xgboost naive_bayes and logistic regression

In [ ]:
import xgboost as xgb

X = pd.read_csv("processed_final_data.csv")
y = pd.read_csv("target_int.csv")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=467)
D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)
xg = xgb()
pprint(xg.get_params())

param = {
    "eta" : 0.25 ,
     "max_depth" : 10 ,
#      "min_child_weight" : 3,
#      "gamma"            : 0.2 ,
#      "colsample_bytree" : 0.4,
    'num_class': 9} 

steps = 100  # The number of training iterations
model = xg.train(param, D_train, steps)
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(y_test, best_preds, average='micro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='micro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))

In [ ]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

### Below modelling has been done using naive bayes and Logistic but SVC gave better result than them

In [ ]:
from sklearn.svm import SVC
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib

X = pd.read_csv("processed_final_data.csv")

y = pd.read_csv("target.csv")

print("##### Data imported #########")
X_norm = (X-X.min())/(X.max()-X.min())
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.05, random_state=467)

svc=SVC(probability=True, kernel='linear')

# Create adaboost classifer object
abc =AdaBoostClassifier(n_estimators=50,base_estimator=svc, learning_rate=1)

print("####### model training started ###########")
# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

# prediction
y_pred = model.predict(X_test)

print("Accuracy:",accuracy_score(y_test, y_pred))

filename = 'support_vector.sav'
joblib.dump(model, filename)


## Random forest with adaboost

Best set of parameter finding

In [ ]:
import numpy as np
import pandas as pd
# from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import train_test_split


X = pd.read_csv("processed_final_data.csv")
target = pd.read_csv("target.csv")
print("data import done  #############")
# X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.05, random_state=467)
#simple modellling
#Random Forest classifier
rf = RFC( random_state = 34)
#classifier.fit(X_train,y_train)
# Number of trees in random forest
n_estimators = [50]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,'max_features': max_features,
               'max_depth': max_depth,'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,'bootstrap': bootstrap}
# pprint(random_grid)
# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
if __name__ == '__main__':
    try:
        # Fit the random search model
        rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, cv = 3, verbose=2, random_state=42, n_jobs = -1)
        rf_random.fit(X , target)
        print(rf_random.best_params_)
    except Exception as err:
        print(err)

modelling with the optimum parameters obtained from the above section for RandomForest with adaboost: This modelling gave best result among all with 63% accuracy 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.externals import joblib


X = pd.read_csv("processed_final_data.csv")
X = X.drop([])
y = pd.read_csv("target.csv")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=467)

random_forest_classifier = RFC(n_estimators = 50,min_samples_split = 2, min_samples_leaf = 4, max_features = 'sqrt', max_depth = 110, bootstrap = True, random_state = 34, n_jobs=1)
abc =AdaBoostClassifier(n_estimators= 100 ,base_estimator=random_forest_classifier, learning_rate=1)

model = abc.fit(X, y)
y_pred = model.predict(X_test)
print("Accuracy:",accuracy_score(y_test, y_pred))

filename = 'adaboost_model.sav'
joblib.dump(model, filename)


## Neural Network

Conv1D and dense layers used : final model gave 62% accuracy on test set.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.externals import joblib
# from __future__ import print_function
from keras.models import Sequential
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from keras.layers import LeakyReLU
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras import backend as K
from keras import optimizers
from time import time
from tensorflow.python.keras.callbacks import TensorBoard

X = pd.read_csv("processed_final_data.csv")
X = X.drop(["breed_int","color_int"], axis =1)
y = pd.read_csv("target_int.csv")
print("######## data imported ########")
X_norm = (X-X.min())/(X.max()-X.min())
X_train, X_test, y_train, y_test = train_test_split(np.asarray(X_norm), np.asarray(y), test_size=0.05, random_state=467)

# The known number of output classes.
num_classes = 9

# Input image dimensions
input_shape = (12,)

# Convert class vectors to binary class matrices. This uses 1 hot encoding.
y_train_binary = keras.utils.to_categorical(y_train, num_classes)
y_test_binary = keras.utils.to_categorical(y_test, num_classes)

X_train = X_train.reshape(45411,12,1)
X_test = X_test.reshape(2391,12,1)

print("######## Defining the neuralnet model ###########")
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model = Sequential()
model.add(Conv1D(160, (3), input_shape=(12,1), activation='relu'))
model.add(Conv1D(160, (3), activation='relu'))
model.add(Conv1D(160, (3), activation='relu'))
model.add(Conv1D(160, (3), activation='relu'))


# model.add(Conv1D(160, (3), activation='relu'))
# model.add(Dropout(0.2))
model.add(MaxPooling1D(2))
# model.add(Conv1D(160, (10), activation='relu'))
# model.add(Conv1D(160, (10), activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))

model.add(Dense(100, activation='relu'))

model.add(Dense(100, activation='relu'))

model.add(Dense(100, activation='relu'))

model.add(Dense(100, activation='relu'))

model.add(Dense(100, activation='relu'))

model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))
# model.add(GlobalAveragePooling1D())
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))
# model.add(Flatten())
# model.add(Dense(64, activation='softmax'))
# model.add(Dense(num_classes, activation='softmax'))
tensorboard = TensorBoard(log_dir = "log/{}".format(time()))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=sgd,
              metrics=['accuracy'])

model.summary()

batch_size = 200
epochs = 10
print("########### model training ##############")
model = model.fit(X_train, y_train_binary,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test_binary), callbacks=[tensorboard])
model.evaluate(X_test, Y_test)

filename = 'neural_net_model.sav'
joblib.dump(model, filename)


## Finding output on Test set

In [ ]:
from sklearn.externals import joblib
import pandas as pd

#load processed test data
df = pd.read_csv("modified_test_data.csv")
loaded_model = joblib.load("adaboost_model.sav")
X = df.drop(["animal_id_outcome"], axis =1)
y_pred = loaded_model.predict(X)
array = df["animal_id_outcome"].tolist()
result = pd.DataFrame({"animal_id_outcome":array,"outcome_type":y_pred})

result.to_csv("result.csv", index = False)